# 使用 Featuretools 进行自动化特征工程

问题：我们有一组截止时间和标签——在标签时间表中——需要为每个标签构建相关特征，并且只使用截止时间之前的数据。传统上，我们会手动完成这项工作，这是一项繁琐且容易出错的过程，使得开发可用的机器学习解决方案变得非常困难。

解决方案：使用 Featuretools 中实现的自动化特征工程，从关系型数据集中构建数百或数千个相关特征，并提供可重用的框架，同时自动根据截止时间过滤数据。这种方法克服了手动特征工程的局限性，让我们能够在更短的时间内构建更好的预测模型。

特征工程的一般过程如下所示：

![jupyter](pictures/特征工程图.PNG)

目前，使用多个相关表进行自动化特征工程的唯一选择是 Featuretools，这是一个开源的 Python 库。

在本笔记本中，我们将使用 Featuretools 开发一个自动化特征工程工作流，用于客户流失数据集。最终的结果是一个函数，它接收一个数据集和客户的标签时间，并构建一个特征矩阵，供训练机器学习模型使用。由于我们已经将数据分成了独立的子集（在“数据分区”部分），因此我们将能够使用 PySpark 在 Spark 中并行应用这个函数到所有分区。

In [1]:
import pandas as pd 
import numpy as np

import featuretools as ft

# 显示每一行的输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N_PARTITIONS = 1000

In [2]:
PARTITION = '50'
BASE_DIR = 'data/partitions/'
PARTITION_DIR = BASE_DIR + 'p' + PARTITION

In [3]:
members = pd.read_csv(f'{PARTITION_DIR}/members.csv',  
                      dtype = {'gender': 'category'})

trans = pd.read_csv(f'{PARTITION_DIR}/transactions.csv',
                   parse_dates=['transaction_date', 'membership_expire_date'])

logs = pd.read_csv(f'{PARTITION_DIR}/logs.csv', parse_dates = ['date'])

cutoff_times = pd.read_csv(f'{PARTITION_DIR}/MS-31_labels.csv', parse_dates = ['cutoff_time'])

这三张数据表由以下模式表示。

![jupyter](../pictures/自动特征工程图.PNG)

这个模式包含了在Featuretools中执行自动特征工程所需的所有领域知识。

# 定义实体和实体集

使用Featuretools的第一步是创建一个实体集并将所有实体（表）添加到其中。实体集是一种数据结构，用于存储表格及它们之间的关系。这使得在处理多个关系表格的问题时，更容易跟踪所有的数据。

In [5]:
import featuretools as ft

# 创建一个空的实体集
es = ft.EntitySet(id = 'customers')

**实体 (Entities)**

在从数据帧 (dataframe) 创建实体时，我们需要确保包括：

*   The ``index`` 如果存在索引或者为创建的索引指定名称。这是每个观测值的唯一标识符。
*   ``make_index = True`` 如果没有索引，我们需要在 ``index`` 下提供一个名称并将其设置为 ``True``。
*   ``time_index`` 如果存在。这是行中信息变得已知的时间。Featuretools 将使用 ``time_index`` 和 ``cutoff_time`` 来为每个标签创建有效的特征。
*   ``variable_types``。 在某些情况下，我们的数据将具有变量，我们需要为其指定类型。一个例子是表示为浮点数的布尔值。 这可以防止 Featuretools 创建诸如 True/False 变量的 ``min`` 或 ``max`` 之类的特征。

对于这个问题，这些是我们需要的唯一参数。 还有其他参数可以按照[文档 (documentation)](链接)中所示的方式使用。

**会员表 (Members Table)**

`members` 表包含每个客户的基本信息。 此表的重要一点是指定 ``city`` 和 ``registered_via`` 列是离散的分类变量而不是数值变量，并且 ``registration_init_time`` 是 ``time_index``。 ``msno`` 是标识每个客户的唯一索引。

In [6]:
members.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,8hW4+CV3D1oNM0CIsA39YljsF8M3m7g1LAX6AQd3C8I=,4,24,male,3,20141104
1,yhcODfebyTYezE6KAPklcV1us9zdOYJ+7eHS7f/xgoU=,8,37,male,9,20070211
2,sBlgSL0AIq49XsmBQ2KceKZNUyIxT1BwSkN/xYQLGMc=,15,21,male,3,20130208
3,Xy3Au8sZKlEeHBQ+C7ro8Ni3X/dxgrtmx0Tt+jqM1zY=,1,0,NaN,9,20150201
4,NiCu2GVWgT5QZbI85oYRBEDqHUZbzz2azS48jvM+khg=,12,21,male,3,20150212


In [7]:
members['msno'].is_unique

False